In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

# Prepare ML input data

In [2]:
# Read Excel file with all possible features
workshop_features = pd.read_excel("workshop_features.xlsx")

In [3]:
workshop_features

,feature,group,workshop
0,arable_prop,arable,1
1,arable_prop_buff_100,arable,0
2,arable_prop_buff_1000,arable,0
3,arable_prop_buff_500,arable,0
4,area,area,1
...,...,...,...
77,twi_mean,twi,1
78,twi_std,twi,0
79,urban_prop,urban,1
80,water_prop,water,1


In [4]:
# Extract selected features
use_features = workshop_features[workshop_features["workshop"] == 1]["feature"].to_list()

In [5]:
use_features

['arable_prop',
 'area',
 'clay1_mean',
 'forest_prop',
 'grassland_prop',
 'k1_mean',
 'limestone_prop',
 'livestock_density',
 'other_prop',
 'precip_mean',
 'rock1_mean',
 'sand1_mean',
 'silt1_mean',
 'slope_mean',
 'soc1_mean',
 'temp_mean',
 'twi_mean',
 'urban_prop',
 'water_prop',
 'wetland_prop']

In [6]:
# Read ML input file with all possible features
ml_input = pd.read_csv("tn_ml_input.csv", sep=",")

In [7]:
ml_input

,site_code,obs_year,parameter,obs_value,arable_prop,arable_prop_buff_100,arable_prop_buff_1000,arable_prop_buff_500,area,awc1_min,...,tri_max,tri_mean,tri_std,twi_min,twi_max,twi_mean,twi_std,urban_prop,water_prop,wetland_prop
0,SJA0088000,2016,TN,1.138,0.122,0.152,0.150,0.178,704297025.0,0.161,...,2.977,0.085,0.108,2.262,15.093,9.851,1.138,0.012,0.004,0.084
1,SJA0088000,2017,TN,1.175,0.122,0.152,0.150,0.178,704297025.0,0.161,...,2.977,0.085,0.108,2.262,15.093,9.851,1.138,0.012,0.004,0.084
2,SJA0106000,2016,TN,5.967,0.428,0.307,0.380,0.368,26553800.0,0.187,...,1.517,0.070,0.072,3.415,13.996,10.252,1.046,0.018,0.000,0.021
3,SJA0106000,2017,TN,4.175,0.428,0.307,0.380,0.368,26553800.0,0.187,...,1.517,0.070,0.072,3.415,13.996,10.252,1.046,0.018,0.000,0.021
4,SJA0106000,2018,TN,4.225,0.428,0.307,0.380,0.368,26553800.0,0.187,...,1.517,0.070,0.072,3.415,13.996,10.252,1.046,0.018,0.000,0.021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,SJB3502000,2019,TN,5.347,0.579,0.460,0.498,0.471,18557575.0,0.174,...,1.723,0.076,0.095,3.264,14.062,10.334,1.200,0.023,0.003,0.025
465,SJB3502000,2020,TN,5.596,0.579,0.460,0.498,0.471,18557575.0,0.174,...,1.723,0.076,0.095,3.264,14.062,10.334,1.200,0.023,0.003,0.025
466,SJB3503000,2020,TN,8.225,0.537,0.316,0.523,0.453,112294125.0,0.174,...,2.764,0.099,0.111,2.279,15.563,10.033,1.392,0.028,0.002,0.014
467,SJB3510000,2019,TN,8.090,0.475,0.271,0.573,0.497,29300575.0,0.174,...,2.757,0.134,0.201,2.321,14.517,9.690,1.718,0.025,0.004,0.096


In [8]:
# Create list of columns to drop from ML input
drop_cols = []
for col in ml_input.columns:
    if col not in use_features and col not in ["site_code", "obs_year", "obs_value"]:
        drop_cols.append(col)

In [9]:
# Drop unnecessary columns
ml_input = ml_input.drop(drop_cols, axis=1)

In [10]:
# Remove rows with missing values
ml_input = ml_input[~(ml_input.isnull()).any(axis=1)].reset_index(drop=True)

In [11]:
# Add observation ID based on index
ml_input.insert(0, "obs_id", ml_input.index + 1)

In [12]:
ml_input

,obs_id,site_code,obs_year,obs_value,arable_prop,area,clay1_mean,forest_prop,grassland_prop,k1_mean,...,rock1_mean,sand1_mean,silt1_mean,slope_mean,soc1_mean,temp_mean,twi_mean,urban_prop,water_prop,wetland_prop
0,1,SJA0088000,2016,1.138,0.122,704297025.0,21.031,0.706,0.035,59.083,...,4.072,65.292,13.677,0.875,8.971,6.524,9.851,0.012,0.004,0.084
1,2,SJA0088000,2017,1.175,0.122,704297025.0,21.031,0.706,0.035,59.083,...,4.072,65.292,13.677,0.875,8.971,6.495,9.851,0.012,0.004,0.084
2,3,SJA0106000,2016,5.967,0.428,26553800.0,25.857,0.491,0.025,16.544,...,5.801,55.326,18.817,0.813,8.528,6.231,10.252,0.018,0.000,0.021
3,4,SJA0106000,2017,4.175,0.428,26553800.0,25.857,0.491,0.025,16.544,...,5.801,55.326,18.817,0.813,8.528,6.133,10.252,0.018,0.000,0.021
4,5,SJA0106000,2018,4.225,0.428,26553800.0,25.857,0.491,0.025,16.544,...,5.801,55.326,18.817,0.813,8.528,7.025,10.252,0.018,0.000,0.021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,461,SJB3502000,2019,5.347,0.579,18557575.0,25.775,0.322,0.032,40.267,...,8.719,57.976,16.249,0.924,8.993,6.842,10.334,0.023,0.003,0.025
461,462,SJB3502000,2020,5.596,0.579,18557575.0,25.775,0.322,0.032,40.267,...,8.719,57.976,16.249,0.924,8.993,7.783,10.334,0.023,0.003,0.025
462,463,SJB3503000,2020,8.225,0.537,112294125.0,22.755,0.304,0.083,14.110,...,9.007,55.590,21.654,1.292,7.390,7.684,10.033,0.028,0.002,0.014
463,464,SJB3510000,2019,8.090,0.475,29300575.0,21.450,0.303,0.082,24.142,...,8.858,59.238,19.312,1.822,8.315,6.743,9.690,0.025,0.004,0.096


In [13]:
ml_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   obs_id             465 non-null    int64  
 1   site_code          465 non-null    object 
 2   obs_year           465 non-null    int64  
 3   obs_value          465 non-null    float64
 4   arable_prop        465 non-null    float64
 5   area               465 non-null    float64
 6   clay1_mean         465 non-null    float64
 7   forest_prop        465 non-null    float64
 8   grassland_prop     465 non-null    float64
 9   k1_mean            465 non-null    float64
 10  limestone_prop     465 non-null    float64
 11  livestock_density  465 non-null    float64
 12  other_prop         465 non-null    float64
 13  precip_mean        465 non-null    float64
 14  rock1_mean         465 non-null    float64
 15  sand1_mean         465 non-null    float64
 16  silt1_mean         465 non

In [14]:
# Write to CSV
ml_input.to_csv("agile2024_tn_ml_input.csv", sep=",", index=False)

# Prepare spatial data

In [15]:
# Extract site codes
site_codes = list(ml_input["site_code"].unique())

In [16]:
# Read observation site points
sites = gpd.read_file("D:/est_water_qual/data/sites.gpkg")

# Extract sites used in the workshop
sites = sites[sites["site_code"].isin(site_codes)].reset_index(drop=True)
display(sites)

# Write to GPKG
sites.to_file("agile2024_sites.gpkg", index=False)

,site_code,x,y,wb_code,geometry
0,SJA8127000,696315.0,6546937.0,VEE1061300,POINT (696315.000 6546937.000)
1,SJA9900000,669868.0,6591973.0,VEE1068200,POINT (669868.000 6591973.000)
2,SJA3956000,636008.0,6603086.0,VEE1074600,POINT (636008.000 6603086.000)
3,SJA1934000,700294.0,6592517.0,VEE1067000,POINT (700294.000 6592517.000)
4,SJA7837000,520653.0,6588232.0,VEE1094500,POINT (520653.000 6588232.000)
...,...,...,...,...,...
234,SJB3503000,633230.0,6585334.0,VEE1074600,POINT (633230.000 6585334.000)
235,SJA3731000,698754.0,6586118.0,VEE1067300,POINT (698754.000 6586118.000)
236,SJA0813000,619933.0,6581023.0,VEE1077900,POINT (619933.000 6581023.000)
237,SJA8884000,551221.0,6591443.0,VEE1089200,POINT (551221.000 6591443.000)


In [17]:
# Read observation site catchments
catchments = gpd.read_file("D:/est_water_qual/data/site_catchments.gpkg")

# Extract catchments used in the workshop
catchments = catchments[catchments["site_code"].isin(site_codes)].reset_index(drop=True)

# Convert geometry to MultiPolygon
new_geoms = []
for geom in catchments["geometry"]:
    if not isinstance(geom, MultiPolygon):
        new_geoms.append(MultiPolygon([geom]))
    else:
        new_geoms.append(geom)
catchments["geometry"] = new_geoms
display(catchments)

# Write to GPKG
catchments.to_file("agile2024_catchments.gpkg", index=False)

,site_code,wb_code,geometry
0,SJA9316000,VEE1080600,"MULTIPOLYGON (((585865.000 6587725.000, 585875..."
1,SJA6880000,VEE1079200,"MULTIPOLYGON (((597930.000 6606720.000, 597930..."
2,SJA8358000,VEE1085000,"MULTIPOLYGON (((599250.000 6568615.000, 599250..."
3,SJA9895000,VEE1079200,"MULTIPOLYGON (((629535.000 6566175.000, 629535..."
4,SJA6180000,VEE1083500,"MULTIPOLYGON (((578040.000 6568635.000, 578040..."
...,...,...,...
234,SJA0680000,VEE1173500,"MULTIPOLYGON (((427385.000 6469925.000, 427385..."
235,SJA9031000,VEE1072900,"MULTIPOLYGON (((642545.000 6592930.000, 642545..."
236,SJA5143000,VEE1084200,"MULTIPOLYGON (((602205.000 6565055.000, 602205..."
237,SJA0135000,VEE1083500,"MULTIPOLYGON (((582165.000 6565725.000, 582165..."
